# AIPI 530 Milk Bank Project

### Team Members: Bryce Whitney, Bruno Valan, Yilun Wu, Andrew Bonafede, Zenan Chen

## Imports

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date
from bs4 import BeautifulSoup
import requests

from pyomo.environ import ConcreteModel, SolverFactory
from pyomo.environ import Set, Constraint, Objective, Param, Var
from pyomo.environ import Binary
from pyomo.environ import minimize, maximize
from pyomo.environ import value

import warnings
warnings.filterwarnings('ignore')


## Load and Clean / Aggregate the Data

### Pre-processing Functions

#### Loading data

In [2]:
def load_data(filepath):
    df = pd.read_excel(filepath, engine='openpyxl')
    return df

#### Cleaning data

In [3]:
def clean_data(data):
    # Drop columns with null values
    data_cleaned = data.dropna(inplace=False)

    # Clean up Oz column 
    data_cleaned.rename(columns = {'#Oz':'Oz'}, inplace = True)
    data_cleaned['Oz'] = data_cleaned['Oz'].replace(regex=[r'\D+'], value="")
    data_cleaned['Oz'] = round(data_cleaned['Oz'].astype(float), 2)
    
    # Clean number of shipments column
    data_cleaned['Number of Shipments'] = pd.to_numeric(data_cleaned['Number of Shipments'], errors = 'coerce')
    data_cleaned.dropna(inplace=True)
    data_cleaned['Number of Shipments'] = data_cleaned['Number of Shipments'].astype(int)
    data_cleaned['Number of Shipments'] = data_cleaned['Number of Shipments'].fillna(1)
    
    # Add pounds column
    data_cleaned['Pounds'] = round(data_cleaned['Oz'].astype(float) / 16, 2)
    
    # Determine average number of Pounds/Oz per shipment
    data_cleaned['Pounds_per_Shipment'] = round(data_cleaned['Pounds'] / data_cleaned['Number of Shipments'], 2)
    data_cleaned['Oz_per_Shipment'] = round(data_cleaned['Oz'] / data_cleaned['Number of Shipments'], 2)
    
    # Create a feature capturing the number of months since the last donation
    data_cleaned['DOLD'] = data_cleaned['DOLD'].replace('10/0/3/2019', '10/3/2019')
    data_cleaned['Current_Date'] = pd.to_datetime(date.today())
    data_cleaned['Months_since_last_donation'] = ((pd.to_datetime(data_cleaned['Current_Date']).dt.year - pd.to_datetime(data_cleaned['DOLD']).dt.year) * 12) + (pd.to_datetime(data_cleaned['Current_Date']).dt.month - pd.to_datetime(data_cleaned['DOLD']).dt.month)
    
    # Convert Dates to pd.Datetime
    data_cleaned['DOFD'] = pd.to_datetime(data_cleaned['DOFD'])
    data_cleaned['DOLD'] = pd.to_datetime(data_cleaned['DOLD'])
    
    # Strip the origin column
    data_cleaned['Origin'] = data_cleaned['Origin'].str.strip()  
    
    # Add column to track if it was dropped off or not
    data_cleaned['droppedOff'] = data_cleaned['Origin'] != 'Shipped from Donor'
    
    # Return the cleaned data
    return data_cleaned

#### Match locations with zip code and latitude & longitude

In [4]:
def generate_zip_data(data):
    # Get unique zipcodes
    zipcode_list = data['Zip code'].unique()

    # Scrape the coordinates
    latitude = []
    longitude = []
    
    for zip_code in zipcode_list:
        url = 'https://www.zipdatamaps.com/{}'.format(zip_code)
        html = requests.get(url)
        Soup = BeautifulSoup(html.content , 'html.parser')
        table = Soup.find(attrs={'class': "table table-striped table-bordered table-hover table-condensed"})
        
        data = [] 
        for i in table.find_all('tr'):
            data.append([j.text for j in i.find_all('td')])
        coord = data[-1][1:]
        
        lat, long = coord[0].split(',')
        latitude.append(lat)
        longitude.append(long)

    # Create the dataframe
    zipcode_df = pd.DataFrame({
        "Zip code": zipcode_list,
        "Latitude": latitude,
        "Longitude":longitude
    })
        
    # Return the zipcode dataframe
    return zipcode_df

### Load and clean/aggregate the data based on previous pre-processing functions

In [5]:
# Load and clean the data
path_to_data = os.path.join(os.getcwd(), 'NYMB_updates.xlsx')
data = load_data(path_to_data)
data_cleaned = clean_data(data)

# Get coordinates for zipcodes
zipcode_df = generate_zip_data(data)

# Merge the datasets
data_cleaned = pd.merge(data_cleaned, zipcode_df, how='inner', on='Zip code')

# Reorganize the columns
data_cleaned = data_cleaned[['DOFD', 'DOLD', 'Neighborhood', 'Latitude', 'Longitude', 'Number of Shipments', 'Oz', 'Oz_per_Shipment', 'droppedOff']]

data_agg = data_cleaned.copy()

# If DOFD != DOLD, use number of shipments to estimate the dates they shipped
# Assume Equally spaced shipments because we have nothing else to go off of
# If DOLD comes before DOFD, set them equal
data_agg.loc[(data_agg['DOLD'] - data_agg['DOFD']).dt.days < 0, 'DOLD'] = data_agg.loc[(data_agg['DOLD'] - data_agg['DOFD']).dt.days < 0, 'DOFD']

# Calculate the dates in the middle
all_dates = []
for i in range(data_agg.shape[0]):
    periods = data_agg.loc[i, 'Number of Shipments']
    dates = pd.to_datetime(pd.date_range(data_agg.iloc[i, :]['DOFD'], data_agg.iloc[i, :]['DOLD'], periods=periods).to_list())
    all_dates.append(dates)
    
data_agg['Dates'] = all_dates
data_agg = data_agg[['DOFD', 'DOLD', 'Dates', 'Neighborhood', 'Latitude', 'Longitude', 'Number of Shipments', 'Oz', 'droppedOff']]

# Adjust the Oz and Number of shipments based on the length of the new Dates column
data_agg['Oz'] = data_agg['Oz'] / data_agg['Number of Shipments']

# Explode the dataframe on the dates column
data_agg = data_agg.explode('Dates')
data_agg = data_agg.drop(columns=['DOFD', 'DOLD', 'Number of Shipments', 'droppedOff'], inplace=False)

data_agg

,Dates,Neighborhood,Latitude,Longitude,Oz
0,2022-08-02,Battery Park,40.70969600,-74.02023300,247.000000
1,2021-04-09,Battery Park,40.70969600,-74.02023300,301.500000
1,2021-05-07,Battery Park,40.70969600,-74.02023300,301.500000
2,2022-02-24,Battery Park,40.70969600,-74.02023300,493.840000
3,2019-04-10,Battery Park,40.71971300,-74.01464900,198.666667
...,...,...,...,...,...
252,2022-03-01,Washington Heights (upper west side),40.85814000,-73.92921000,276.000000
252,2022-04-22,Washington Heights (upper west side),40.85814000,-73.92921000,276.000000
252,2022-06-13,Washington Heights (upper west side),40.85814000,-73.92921000,276.000000
252,2022-08-04,Washington Heights (upper west side),40.85814000,-73.92921000,276.000000


## Computations & Data Processing

### Compute average biweekly donations

In [6]:
# Calculate the number of biweekly groups
num_biweekly_groups = pd.date_range(min(data_agg['Dates']), max(data_agg['Dates']), freq='2W')
num_biweekly_groups = len(num_biweekly_groups) - 1

# Calculate the biweekly total donations for each neighborhhod
biweekly_total_donations = data_agg.groupby(['Neighborhood', pd.Grouper(key='Dates', freq='SM')]).sum().reset_index()
biweekly_total_donations = biweekly_total_donations.sort_values(['Dates', 'Neighborhood'])
biweekly_total_donations[biweekly_total_donations['Neighborhood'] == 'Battery Park']

# Calculate Average Biweekly Donations for each neighborhood
avg_biweekly_donations = biweekly_total_donations.groupby('Neighborhood').sum().reset_index()
avg_biweekly_donations['Oz'] = avg_biweekly_donations['Oz'] / num_biweekly_groups
avg_biweekly_donations = avg_biweekly_donations.set_index('Neighborhood')

avg_biweekly_donations

,Oz
Neighborhood,
Battery Park,15.870976
Bowling Green,12.862073
Chelsea,50.354207
City Hall,3.439024
East Village,50.807073
Hamilton Heights (upper west side),14.962134
Harlem,48.150244
Inwood,13.548049
Lower East Side,3.507378


### Compute shipping costs

In [7]:
# Calculate the average amount that was shipped
total_shipped = data_cleaned[data_cleaned['droppedOff'] == False]['Oz'].sum()
total_donated = data_cleaned['Oz'].sum()
percentage_shipped = total_shipped / total_donated

# Compute the shipping cost based on pounds
def compute_shipping_cost_pounds(weight_in_pounds=0):
    weight_limit = range(2, 18)
    shipping_cost = [41, 45.76, 49.92, 50.34, 57.70, 58.44, 58.70, 59.49, 60.22, 70.75, 72.42, 72.68, 73.11, 73.37, 78.14, 82.62, 83.28]
    for i in range(len(weight_limit)):
        if weight_in_pounds <= weight_limit[i]: return shipping_cost[i]

# Compute the shipping cost based on ounces   
def compute_shipping_cost_ounces(weight_in_ounces):
    weight_in_pounds = weight_in_ounces / 16
    return compute_shipping_cost_pounds(weight_in_pounds)

# Compute shipping costs
avg_biweekly_donations['ShippingCostPortion'] = avg_biweekly_donations['Oz'].apply(lambda x: compute_shipping_cost_ounces(percentage_shipped * x))
avg_biweekly_donations['ShippingCostAll'] = avg_biweekly_donations['Oz'].apply(lambda x: compute_shipping_cost_ounces(x))

# Shipping cost dictionaries for building constraints
shipping_costs_all = avg_biweekly_donations.to_dict()['ShippingCostAll']
shipping_cost_portion = avg_biweekly_donations.to_dict()['ShippingCostPortion']

# Add latitude back
lats = []
longs = []
neighborhoods = avg_biweekly_donations.index.values
for n in neighborhoods:
    lats.append(np.mean(data_cleaned[data_cleaned['Neighborhood'] == n]['Latitude'].astype(float)))
    longs.append(np.mean(data_cleaned[data_cleaned['Neighborhood'] == n]['Longitude'].astype(float)))
    
# Round to 6 decimal places
lats = np.round(lats, 6)
longs = np.round(longs, 6)

# Add to the dataFrame
avg_biweekly_donations['Latitude'] = lats
avg_biweekly_donations['Longitude'] = longs

# Save the dataframe to a csv
avg_biweekly_donations.to_csv('ModelData.csv')

# save the values to a list
latitudes = avg_biweekly_donations.to_dict()['Latitude']
longitudes = avg_biweekly_donations.to_dict()['Longitude']

avg_biweekly_donations

,Oz,ShippingCostPortion,ShippingCostAll,Latitude,Longitude
Neighborhood,,,,,
Battery Park,15.870976,41.00,41.00,40.714704,-74.017441
Bowling Green,12.862073,41.00,41.00,40.694938,-74.016926
Chelsea,50.354207,45.76,49.92,40.746469,-74.002162
City Hall,3.439024,41.00,41.00,40.708615,-74.002016
East Village,50.807073,45.76,49.92,40.730173,-73.984950
Hamilton Heights (upper west side),14.962134,41.00,41.00,40.824878,-73.950087
Harlem,48.150244,45.76,49.92,40.801216,-73.945457
Inwood,13.548049,41.00,41.00,40.870817,-73.923073
Lower East Side,3.507378,41.00,41.00,40.713884,-73.985923


### Compute distances between neighborhoods

In [8]:
# Calculate a distance matrix between neighborhoods in miles
RADIAN_CONVERSION = 1 / 57.29577951
distances = np.zeros(shape=(len(neighborhoods), len(neighborhoods)))

for i, n1 in enumerate(neighborhoods):
    for j, n2 in enumerate(neighborhoods):
        
        # Convert Lat and Long to radians
        lat1 = avg_biweekly_donations[avg_biweekly_donations.index == n1]['Latitude'].values * RADIAN_CONVERSION
        lat2 = avg_biweekly_donations[avg_biweekly_donations.index == n2]['Latitude'].values * RADIAN_CONVERSION
        long1 = avg_biweekly_donations[avg_biweekly_donations.index == n1]['Longitude'].values * RADIAN_CONVERSION
        long2 = avg_biweekly_donations[avg_biweekly_donations.index == n2]['Longitude'].values * RADIAN_CONVERSION
        
        d = 3963.0 * np.arccos((np.sin(lat1) * np.sin(lat2)) + np.cos(lat1) * np.cos(lat2) * np.cos(long2 - long1))
        distances[i, j] = d
        
distances_df = pd.DataFrame(distances, columns=neighborhoods, index=neighborhoods)
distances_df

,Battery Park,Bowling Green,Chelsea,City Hall,East Village,Hamilton Heights (upper west side),Harlem,Inwood,Lower East Side,Meatpacking District,...,Morningside Heights,Murray Hill,NOHO,Roosevelt Island,Stuyvesant Park,Tribeca,Wall Street,Washington Heights (upper west side),upper east side,upper west side
Battery Park,0.000000,1.367429,2.338503,0.911811,2.011384,8.397592,7.073161,11.874954,1.653365,1.356046,...,7.504573,3.068100,1.266021,4.724504,2.595460,0.478016,0.752692,10.284676,5.229020,5.078966
Bowling Green,1.367429,0.000000,3.647327,1.227266,2.958039,9.645697,8.249957,13.120599,2.088050,2.693150,...,8.736630,4.148886,2.338829,5.731239,3.614430,1.616817,0.890920,11.532902,6.335928,6.334677
Chelsea,2.338503,3.647327,0.000059,2.618274,1.443662,6.070446,4.812618,9.545577,2.409185,0.990432,...,5.189464,1.326767,1.462983,2.839656,1.298557,2.033027,2.808816,7.955328,3.112310,2.757261
City Hall,0.911811,1.227266,2.618274,0.000000,1.738910,8.489268,7.057294,11.956517,0.919089,1.778792,...,7.571558,2.943729,1.190535,4.506062,2.399916,0.722266,0.383348,10.372820,5.125556,5.198059
East Village,2.011384,2.958039,1.443662,1.738910,0.000000,6.800249,5.331637,10.253291,1.127822,1.349885,...,5.874774,1.216922,0.754554,2.777262,0.663280,1.539131,2.072036,8.675772,3.388204,3.558516
Hamilton Heights (upper west side),8.397592,9.645697,6.070446,8.489268,6.800249,0.000059,1.654489,3.477662,7.903342,7.060717,...,0.932404,5.588663,7.313812,4.388720,6.147742,8.041870,8.764836,1.887897,3.566561,3.318752
Harlem,7.073161,8.249957,4.812618,7.057294,5.331637,1.654489,0.000000,4.954581,6.401790,5.783457,...,0.815177,4.116087,5.912855,2.770614,4.669120,6.679445,7.359991,3.410555,1.980809,2.128693
Inwood,11.874954,13.120599,9.545577,11.956517,10.253291,3.477662,4.954581,0.000000,11.342624,10.536008,...,4.385346,9.037236,10.786038,7.712345,9.595284,11.518875,12.237867,1.590353,6.935368,6.796347
Lower East Side,1.653365,2.088050,2.409185,0.919089,1.127822,7.903342,6.401790,11.342624,0.000000,1.866936,...,6.974429,2.316437,1.025647,3.746556,1.758167,1.243116,1.302059,9.771389,4.433530,4.683474
Meatpacking District,1.356046,2.693150,0.990432,1.778792,1.349885,7.060717,5.783457,10.536008,1.866936,0.000059,...,6.177625,1.974636,0.850379,3.627125,1.654733,1.108132,1.897408,8.945752,4.016234,3.746084


### Compute and add geographical constraints

In [9]:
constraint_file = 'geo_constraints.txt'

distance_threshold = 1 # In miles

with open(constraint_file, 'w+') as file:
    count = 0
    for i in range(len(distances_df.index) - 1):
        for j in range(i+1, len(distances_df.columns)): # Doesn't check neighborhood combinations twice
            # Get the neighborhood
            n1 = distances_df.index[i]
            n2 = distances_df.columns[j]
            
            # Get the distance between them
            dist = distances_df.iloc[i, j]
            
            # If they are too close, write the constraint
            if(dist < distance_threshold):
                count += 1
                file.write(f'def GeographicConstraint_rule{count}(m):\n')
                file.write(f'\treturn m.Depo["{n1}"] + m.Depo["{n2}"] <= 1\n') 
                file.write(f'm.GeographicConstraint{count} = Constraint(rule=GeographicConstraint_rule{count})\n\n')           
print('Number of Constraints: ', count)

Number of Constraints:  20


## Model with Volunteer
### Formulation
#### Important Notes:
- Fixed startup = $500
- Cost of shipment in Manhattan is the same, entirely dependent on weight of the shipment. **Consolidation is Key**. *Want to Attack places with the most donations ounces*
#### Decision Variables: 
- Depo[n]: Binary Decision variable for each neighborhood of whether or not to place a milk bank 
#### Objective: Minimize Cost
- Total_Cost = Creation_Cost + Shipping Costs
#### Constraints
- Between 2 - 5 Depos
#### Assumptions we are making
- People will donate roughly the same whether they mail it in or drop it off
- Neighborhood donation rate will remain approximately the same in the future
- Percentage of people who drop off will remain roughly the same
- Assume shipping FedEx overnight
- Equally spaced shipments for people who have multiple on different dates



### Implementation & Solving

In [10]:
m = ConcreteModel()

# Sets
m.NEIGHBORHOODS = Set(initialize=neighborhoods)

# Inputs
m.min_depos = Param(initialize=2)
m.max_depos = Param(initialize=5)
m.depo_creation_cost = Param(initialize=500)

m.min_depo_distance = Param(initialize=distance_threshold)
m.shipping_costs_all = Param(m.NEIGHBORHOODS, initialize=shipping_costs_all)
m.shipping_costs_portion = Param(m.NEIGHBORHOODS, initialize=shipping_cost_portion)

# Decision Variables
m.Depo = Var(m.NEIGHBORHOODS, domain=Binary)

# Constraints
def EnoughDepos_rule(m):
    return sum(m.Depo[n] for n in m.NEIGHBORHOODS) >= m.min_depos
m.EnoughDepos_constraint = Constraint(rule=EnoughDepos_rule)

def NotTooManyDepos_rule(m):
    return sum(m.Depo[n] for n in m.NEIGHBORHOODS) <= m.max_depos
m.NotTooManyDepos_rule = Constraint(rule=NotTooManyDepos_rule)

################################
# START GEOGRAPHIC CONSTRAINTS #
################################

def GeographicConstraint_rule1(m):
	return m.Depo["Battery Park"] + m.Depo["City Hall"] <= 1
m.GeographicConstraint1 = Constraint(rule=GeographicConstraint_rule1)

def GeographicConstraint_rule2(m):
	return m.Depo["Battery Park"] + m.Depo["Tribeca"] <= 1
m.GeographicConstraint2 = Constraint(rule=GeographicConstraint_rule2)

def GeographicConstraint_rule3(m):
	return m.Depo["Battery Park"] + m.Depo["Wall Street"] <= 1
m.GeographicConstraint3 = Constraint(rule=GeographicConstraint_rule3)

def GeographicConstraint_rule4(m):
	return m.Depo["Bowling Green"] + m.Depo["Wall Street"] <= 1
m.GeographicConstraint4 = Constraint(rule=GeographicConstraint_rule4)

def GeographicConstraint_rule5(m):
	return m.Depo["Chelsea"] + m.Depo["Meatpacking District"] <= 1
m.GeographicConstraint5 = Constraint(rule=GeographicConstraint_rule5)

def GeographicConstraint_rule6(m):
	return m.Depo["City Hall"] + m.Depo["Lower East Side"] <= 1
m.GeographicConstraint6 = Constraint(rule=GeographicConstraint_rule6)

def GeographicConstraint_rule7(m):
	return m.Depo["City Hall"] + m.Depo["Tribeca"] <= 1
m.GeographicConstraint7 = Constraint(rule=GeographicConstraint_rule7)

def GeographicConstraint_rule8(m):
	return m.Depo["City Hall"] + m.Depo["Wall Street"] <= 1
m.GeographicConstraint8 = Constraint(rule=GeographicConstraint_rule8)

def GeographicConstraint_rule9(m):
	return m.Depo["East Village"] + m.Depo["NOHO"] <= 1
m.GeographicConstraint9 = Constraint(rule=GeographicConstraint_rule9)

def GeographicConstraint_rule10(m):
	return m.Depo["East Village"] + m.Depo["Stuyvesant Park"] <= 1
m.GeographicConstraint10 = Constraint(rule=GeographicConstraint_rule10)

def GeographicConstraint_rule11(m):
	return m.Depo["Hamilton Heights (upper west side)"] + m.Depo["Morningside Heights"] <= 1
m.GeographicConstraint11 = Constraint(rule=GeographicConstraint_rule11)

def GeographicConstraint_rule12(m):
	return m.Depo["Harlem"] + m.Depo["Morningside Heights"] <= 1
m.GeographicConstraint12 = Constraint(rule=GeographicConstraint_rule12)

def GeographicConstraint_rule13(m):
	return m.Depo["Meatpacking District"] + m.Depo["NOHO"] <= 1
m.GeographicConstraint13 = Constraint(rule=GeographicConstraint_rule13)

def GeographicConstraint_rule14(m):
	return m.Depo["Midtown-East side"] + m.Depo["Murray Hill"] <= 1
m.GeographicConstraint14 = Constraint(rule=GeographicConstraint_rule14)

def GeographicConstraint_rule15(m):
	return m.Depo["Midtown-East side"] + m.Depo["Roosevelt Island"] <= 1
m.GeographicConstraint15 = Constraint(rule=GeographicConstraint_rule15)

def GeographicConstraint_rule16(m):
	return m.Depo["Midtown-West side"] + m.Depo["upper west side"] <= 1
m.GeographicConstraint16 = Constraint(rule=GeographicConstraint_rule16)

def GeographicConstraint_rule17(m):
	return m.Depo["Murray Hill"] + m.Depo["Stuyvesant Park"] <= 1
m.GeographicConstraint17 = Constraint(rule=GeographicConstraint_rule17)

def GeographicConstraint_rule18(m):
	return m.Depo["NOHO"] + m.Depo["Tribeca"] <= 1
m.GeographicConstraint18 = Constraint(rule=GeographicConstraint_rule18)

def GeographicConstraint_rule19(m):
	return m.Depo["Roosevelt Island"] + m.Depo["upper east side"] <= 1
m.GeographicConstraint19 = Constraint(rule=GeographicConstraint_rule19)

def GeographicConstraint_rule20(m):
	return m.Depo["Tribeca"] + m.Depo["Wall Street"] <= 1
m.GeographicConstraint20 = Constraint(rule=GeographicConstraint_rule20)

##############################
# END GEOGRAPHIC CONSTRAINTS #
##############################

# Objective Function
def TotalCost_objective(m):
    # If depo == 0 --> (1 - (m.depo[n] * m.percentage_milk_dropped)) = 1
    # If depo == 1 --> (1 - (m.Depo[n] * m.percentage_milk_dropped)_ = % milk shipped 
    return sum(m.Depo[n]*m.depo_creation_cost + (m.Depo[n] * m.shipping_costs_portion[n] + (1-m.Depo[n])*m.shipping_costs_all[n])*21 for n in m.NEIGHBORHOODS)
m.objective = Objective(rule=TotalCost_objective, sense=minimize)

In [11]:
# Solve the problem
solver = SolverFactory('glpk')
results = solver.solve(m)

# Print the results
print(results['Solver'])
print(m.objective.expr())


- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.014537811279296875

22066.569999999996


### Results

In [12]:
print("Optimal Milk Bank Locations are: ")
for n in m.NEIGHBORHOODS:
    if(value(m.Depo[n] == 1)):
        print(n)        
        
display(avg_biweekly_donations.sort_values('Oz'))

Optimal Milk Bank Locations are: 
upper east side
upper west side


,Oz,ShippingCostPortion,ShippingCostAll,Latitude,Longitude
Neighborhood,,,,,
Wall Street,0.621951,41.00,41.00,40.706151,-74.008565
City Hall,3.439024,41.00,41.00,40.708615,-74.002016
Lower East Side,3.507378,41.00,41.00,40.713884,-73.985923
Midtown-East side,10.427561,41.00,41.00,40.758361,-73.967646
NOHO,10.475610,41.00,41.00,40.725555,-73.997992
Stuyvesant Park,10.862073,41.00,41.00,40.738836,-73.979523
Roosevelt Island,12.347561,41.00,41.00,40.761439,-73.951697
Bowling Green,12.862073,41.00,41.00,40.694938,-74.016926
Inwood,13.548049,41.00,41.00,40.870817,-73.923073
